In [4]:
pip install dash

Note: you may need to restart the kernel to use updated packages.


In [9]:
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import requests

app = dash.Dash(__name__)

def get_weather_data(city_name, api_key):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city_name}&appid={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        return None

def kelvin_to_fahrenheit(temp_kelvin):
    return round((temp_kelvin - 273.15) * 9/5 + 32, 2)

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(
    style={'background-image': 'url("https://source.unsplash.com/1600x900/?weather")', 'backgroundSize': 'cover', 'height': '100vh'},
    children=[
        html.Div([
            html.H1("Weather Dashboard", style={'textAlign': 'center', 'color': 'white'}),
            html.H2("Check Your City Weather Below", style={'textAlign': 'center', 'color': 'white'}),
            dcc.Input(id='city-input', type='text', placeholder='Enter city name', style={'width': '50%', 'margin': '0 auto', 'display': 'block'}),
            html.Button('Submit', id='submit-val', n_clicks=0, style={'margin': '20px auto', 'display': 'block'}),
            html.Div(id='weather-output', style={'textAlign': 'center', 'color': 'white'})
        ])
    ]
)

@app.callback(
    Output('weather-output', 'children'),
    [Input('submit-val', 'n_clicks')],
    [dash.dependencies.State('city-input', 'value')]
)
def update_output(n_clicks, city):
    if n_clicks > 0:
        api_key = "08851d856c48bb69f37f685021b69031"
        city = city.lower()
        weather_data = get_weather_data(city, api_key)
        if weather_data:
            city_name = weather_data['name']
            temperature_kelvin = weather_data['main']['temp']
            temperature_fahrenheit = kelvin_to_fahrenheit(temperature_kelvin)
            humidity = weather_data['main']['humidity']
            weather_description = weather_data['weather'][0]['description']
            return html.Div([
                html.H3(f"City: {city_name}"),
                html.P(f"Temperature: {temperature_fahrenheit} °F"),
                html.P(f"Humidity: {humidity}%"),
                html.P(f"Weather Description: {weather_description}"),
                html.P("Stay tuned for more updates!", style={'fontStyle': 'italic', 'marginTop': '20px', 'color': 'lightgray'})
            ])
        else:
            return html.P("Failed to fetch weather data. Please try again.")
    else:
        return ''

if __name__ == '__main__':
    app.run_server(debug=True)
